In [4]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [9]:
TRAIN_DIR = 'dataset1/train'
TEST_DIR = 'dataset1/test'

In [8]:

def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels
            

In [10]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [12]:

print(train)

                                               image     label
0         dataset1/train\angry\Training_10118481.jpg     angry
1         dataset1/train\angry\Training_10120469.jpg     angry
2         dataset1/train\angry\Training_10131352.jpg     angry
3         dataset1/train\angry\Training_10161559.jpg     angry
4          dataset1/train\angry\Training_1021836.jpg     angry
...                                              ...       ...
28704  dataset1/train\surprise\Training_99916297.jpg  surprise
28705  dataset1/train\surprise\Training_99924420.jpg  surprise
28706  dataset1/train\surprise\Training_99937001.jpg  surprise
28707  dataset1/train\surprise\Training_99951755.jpg  surprise
28708  dataset1/train\surprise\Training_99984132.jpg  surprise

[28709 rows x 2 columns]


In [13]:

test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [14]:

def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale = True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [15]:
train_features = extract_features(train['image'])

  0%|          | 0/28709 [00:00<?, ?it/s]

c:\Users\91970\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [16]:
test_features = extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [17]:
x_train = train_features/255.0
x_test = test_features/255.0

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [20]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [21]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [22]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
#outputLayer
model.add(Dense(7, activation='softmax'))

c:\Users\91970\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [23]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [24]:
model.fit(x=x_train,y=y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 252s 1s/step - accuracy: 0.2290 - loss: 1.8554 - val_accuracy: 0.2591 - val_loss: 1.7857
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.2840 - loss: 1.7458 - val_accuracy: 0.3787 - val_loss: 1.5795
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 270s 1s/step - accuracy: 0.3706 - loss: 1.5973 - val_accuracy: 0.4454 - val_loss: 1.4645
Epoch 4/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - accuracy: 0.4258 - loss: 1.4832 - val_accuracy: 0.4717 - val_loss: 1.3759
Epoch 5/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 271s 1s/step - accuracy: 0.4544 - loss: 1.4042 - val_accuracy: 0.5017 - val_loss: 1.2839
Epoch 6/100
 50/225 ━━━━━━━━━━━━━━━━━━━━ 3:21 1s/step - accuracy: 0.4745 - loss: 1.3655

KeyboardInterrupt: 

In [25]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [26]:
from keras.models import model_from_json

In [27]:
json_file = open('emotiondetector.json','r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

label = ['angry', 'disgust', 'fear', 'happy', 'neural', 'sad', 'surprise']

In [34]:
def ef(image):
    img = load_img(image,grayscale = True)
    feature = np.array(img)
    feature.reshape(1,48,48,1)
    return feature/255.0

In [35]:
image = 'dataset1/train/angry/22.jpg'
print("original  image is of angry")
image = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)

original  image is of angry


c:\Users\91970\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


FileNotFoundError: [Errno 2] No such file or directory: 'dataset1/train/angry/22.jpg'

In [36]:
import os

# Define the paths to the two files
file1_path = 'dataset1/train/angry/22.jpg'
file2_path = 'emotiondetector.json'

# Get the directory paths of the files
file1_directory = os.path.dirname(file1_path)
file2_directory = os.path.dirname(file2_path)

# Compare the directory paths
if file1_directory == file2_directory:
    print("The two files are in the same directory.")
else:
    print("The two files are NOT in the same directory.")

The two files are NOT in the same directory.


In [37]:
import shutil
import os

# Define the paths to the two files
file1_path = 'dataset1/train/angry/22.jpg'
file2_path = 'emotiondetector.json'

# Get the directory path of file2
file2_directory = os.path.dirname(file2_path)

# Move file1 to the directory of file2
shutil.move(file1_path, file2_directory)

print("Moved file to the same directory.")

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [38]:
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
image

'dataset1/train/angry/22.jpg'